In [ ]:
import pandas as pd

def imprimir_encabezados_reales():
    archivo = "capIQReport.xls" # <--- Asegúrate que se llame así
    
    print(f"🔍 Escaneando {archivo}...")
    
    try:
        # Leemos las primeras 15 filas sin asurmir cuál es el encabezado
        df = pd.read_excel(archivo, header=None, nrows=15)
        
        encontrado = False
        
        # Buscamos fila por fila
        for index, row in df.iterrows():
            # Convertimos toda la fila a texto para buscar palabras clave
            texto_fila = str(row.values)
            
            # Si la fila tiene "Ticker" y "EPS", esa es la cabecera
            if "Ticker" in texto_fila and ("EPS" in texto_fila or "Debt" in texto_fila):
                print(f"\n✅ ¡ENCABEZADOS ENCONTRADOS EN FILA {index}!")
                print("👇 COPIA Y PEGA LO SIGUIENTE EN EL CHAT 👇\n")
                print("-" * 60)
                
                # Imprimimos cada columna limpia
                for col in row:
                    # Solo imprimimos si no está vacío y parece un dato relevante
                    if str(col) != 'nan' and ("EPS" in str(col) or "Debt" in str(col)):
                        print(f"COLUMNA: {col}")
                        
                print("-" * 60)
                encontrado = True
                break
        
        if not encontrado:
            print("❌ No encontré ninguna fila que diga 'Ticker' y 'EPS' en las primeras 15 filas.")
            print("Aquí están las primeras filas para que veas qué hay:")
            print(df.head())

    except FileNotFoundError:
        print("❌ ERROR: No encuentro el archivo 'capIQReport.xls' en esta carpeta.")

if __name__ == "__main__":
    imprimir_encabezados_reales()

In [12]:
import pandas as pd
import re
import os
from datetime import datetime, timedelta

def limpiar_capiq_relativo():
    input_file = "capIQReport.xls"
    output_file = "tus_datos_capiq.xlsx"
    
    # --- CONFIGURACIÓN CLAVE ---
    # Pon aquí la fecha APROXIMADA en que descargaste el Excel.
    # El script asumirá que la columna [LTM] corresponde a este trimestre.
    FECHA_REFERENCIA = "2025-11-28" 
    
    print(f"1. 🧹 Leyendo archivo: {input_file}...")
    
    if not os.path.exists(input_file):
        print(f"❌ ERROR: No encuentro '{input_file}'.")
        return

    try:
        # Buscar la cabecera dinámicamente
        df_raw = pd.read_excel(input_file, header=None)
        start_row = 0
        for i, row in df_raw.iterrows():
            row_str = row.astype(str).str.cat()
            if "Ticker" in row_str and ("EPS" in row_str or "Debt" in row_str):
                start_row = i
                break
        
        print(f"   -> Cabecera encontrada en la fila {start_row + 1}")
        df = pd.read_excel(input_file, header=start_row)
        
    except Exception as e:
        print(f"❌ Error leyendo Excel: {e}")
        return

    col_ticker = next((c for c in df.columns if "Ticker" in str(c)), None)
    if not col_ticker:
        print("❌ Error: No encuentro columna Ticker.")
        return

    print("2. 🧮 Calculando fechas basadas en columnas relativas...")
    
    # Convertimos la fecha de referencia a objeto fecha
    ref_date = pd.to_datetime(FECHA_REFERENCIA)
    
    processed_data = []
    
    # Regex para capturar el numero de offset: "LTM - 15" o "Latest Quarter - 3"
    # Grupo 1: El Offset (si existe)
    regex_offset = re.compile(r'\[(?:LTM|Latest Quarter)(?: - (\d+))?\]')

    # Convertir a formato largo
    df_melt = df.melt(id_vars=[col_ticker], var_name='raw_col', value_name='value')
    # Limpiar valores no numéricos
    df_melt['value'] = pd.to_numeric(df_melt['value'], errors='coerce')
    df_melt = df_melt.dropna(subset=['value'])

    count = 0
    for idx, row in df_melt.iterrows():
        header = str(row['raw_col'])
        
        # 1. Identificar Métrica
        metric = None
        if "Normalized Diluted EPS" in header: metric = "eps_norm"
        elif "Total Debt" in header: metric = "total_debt"
        
        if not metric: continue

        # 2. Calcular Fecha
        match = regex_offset.search(header)
        if match:
            # Si captura un numero (ej: 15), es el offset. Si es None, es 0 (LTM actual)
            offset_quarters = int(match.group(1)) if match.group(1) else 0
            
            # CALCULO DE FECHA: Fecha Ref - (Offset * 3 meses)
            # Usamos 91 dias como aprox de un trimestre para ser rápidos
            days_to_subtract = offset_quarters * 91 
            calculated_date = ref_date - timedelta(days=days_to_subtract)
            
            processed_data.append({
                'tic': str(row[col_ticker]).replace('NYSE:', '').replace('NasdaqGS:', ''),
                'date': calculated_date,
                'metric': metric,
                'value': row['value']
            })
            count += 1

    if not processed_data:
        print("❌ ERROR: No se pudieron procesar datos. Revisa el regex.")
        return

    print(f"   -> {count} registros procesados.")

    # Paso 3: Armar tabla final
    df_clean = pd.DataFrame(processed_data)
    
    # Normalizar fechas al fin de mes más cercano (opcional, para que quede bonito)
    df_clean['date'] = df_clean['date'] + pd.offsets.MonthEnd(0)
    
    # Pivotar
    df_final = df_clean.pivot_table(index=['date', 'tic'], columns='metric', values='value').reset_index()
    df_final = df_final.sort_values(['tic', 'date'])
    
    df_final.to_excel(output_file, index=False)
    print("="*40)
    print(f"✅ ¡LISTO! Archivo generado: {output_file}")
    print(f"   Rango de Fechas: {df_final.date.min().date()} a {df_final.date.max().date()}")
    print("="*40)
    print(df_final.head())

if __name__ == "__main__":
    limpiar_capiq_relativo()

1. 🧹 Leyendo archivo: capIQReport.xls...
   -> Cabecera encontrada en la fila 8
2. 🧮 Calculando fechas basadas en columnas relativas...
   -> 960 registros procesados.
✅ ¡LISTO! Archivo generado: tus_datos_capiq.xlsx
   Rango de Fechas: 2022-03-31 a 2025-11-30
metric       date   tic  eps_norm  total_debt
0      2022-03-31  AAPL      4.37    122798.0
30     2022-06-30  AAPL      4.48    119981.0
60     2022-09-30  AAPL      4.47    119691.0
90     2022-12-31  AAPL      4.56    132480.0
120    2023-03-31  AAPL      4.38    111110.0


In [13]:
import pandas as pd
import numpy as np
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl import config_tickers

# ==============================================================================
# FUNCIÓN DE COVARIANZA (Necesaria para el Portfolio Env)
# ==============================================================================
def add_covariance_matrix(df, lookback=252):
    # Aseguramos que date sea columna y no indice
    if 'date' not in df.columns:
        df = df.reset_index()
        
    df = df.sort_values(['date','tic'], ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []
    dates_with_cov = [] 
    
    unique_dates = df.date.unique()

    # Si no hay suficientes datos, devolvemos tal cual para evitar crash
    if len(unique_dates) < lookback:
        return df

    print("   -> Generando matrices de covarianza (esto toma unos segundos)...")
    for i in range(lookback, len(unique_dates)):
        current_date = unique_dates[i]
        data_lookback = df.loc[i-lookback:i, :]
        price_lookback = data_lookback.pivot_table(index='date', columns='tic', values='close')
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values 
        cov_list.append(covs)
        dates_with_cov.append(current_date)

    df_cov = df[df.date.isin(dates_with_cov)].copy()
    cov_dict = dict(zip(dates_with_cov, cov_list))
    df_cov['cov_list'] = df_cov['date'].map(cov_dict)
    
    return df_cov.sort_values(['date', 'tic']).reset_index(drop=True)

# ==============================================================================
# FUNCIÓN PRINCIPAL DE FUSIÓN DE DATOS
# ==============================================================================
def prepare_custom_data():
    print("1. 📉 Descargando Precios + Macro de Yahoo...")
    
    dow_tickers = [t for t in config_tickers.DOW_30_TICKER if t != 'WBA']
    
    # A. PRECIOS (Stocks)
    # Bajamos desde 2018 para tener suficiente historia para el lookback
    df_prices = YahooDownloader(start_date='2018-01-01', 
                                end_date='2024-01-01', 
                                ticker_list=dow_tickers).fetch_data()
    df_prices = df_prices.reset_index(drop=True)
    df_prices['date'] = pd.to_datetime(df_prices['date'])

    # B. MACRO (VIX, Bonos, Oro)
    macro_tickers = ['^VIX', '^TNX', 'GC=F']
    df_macro = YahooDownloader(start_date='2018-01-01', 
                               end_date='2024-01-01', 
                               ticker_list=macro_tickers).fetch_data()
    
    # Procesar Macro: Pivotar para tener columnas limpias
    df_macro = df_macro.pivot(index='date', columns='tic', values='close').reset_index()
    df_macro['date'] = pd.to_datetime(df_macro['date'])
    
    # Renombrar columnas para que sean fáciles de leer
    rename_map = {'^VIX': 'vix', '^TNX': 'us_10y', 'GC=F': 'gold'}
    df_macro.rename(columns=rename_map, inplace=True)
    df_macro = df_macro.ffill().bfill() # Rellenar huecos de festivos

    print("2. 📊 Calculando Indicadores Técnicos...")
    fe = FeatureEngineer(use_technical_indicator=True,
                         tech_indicator_list=['macd', 'rsi_30'], 
                         use_turbulence=False,
                         user_defined_feature=False)
    df_prices = fe.preprocess_data(df_prices)
    if 'date' not in df_prices.columns: df_prices = df_prices.reset_index()

    print("3. 🔗 Fusionando con Datos Fundamentales (Capital IQ)...")
    try:
        # Cargar tu Excel limpio
        df_capiq = pd.read_excel("tus_datos_capiq.xlsx")
        df_capiq['date'] = pd.to_datetime(df_capiq['date'])
        
        # Verificar que existen las columnas necesarias
        if 'tic' in df_capiq.columns and 'date' in df_capiq.columns:
            # MERGE: Unimos precios diarios con datos trimestrales
            # Usamos 'on' date y tic. Solo coincidirán las fechas de cierre de trimestre.
            df_prices = df_prices.merge(df_capiq, on=['date', 'tic'], how='left')
            
            # FORWARD FILL: Aquí ocurre la magia.
            # Rellenamos los NaNs hacia abajo usando el último dato conocido por Ticker.
            fund_cols = ['eps_norm', 'total_debt']
            df_prices[fund_cols] = df_prices.groupby('tic')[fund_cols].ffill()
            
            # Si al principio hay NaNs (antes del primer reporte), rellenar con 0
            df_prices[fund_cols] = df_prices[fund_cols].fillna(0)

            # CALCULAR EL "SUPER INDICADOR": P/E RATIO DIARIO
            # (Sumamos 0.01 para evitar división por cero)
            df_prices['pe_ratio_daily'] = df_prices['close'] / (df_prices['eps_norm'] + 0.01)
            
            print("   ✅ ¡Datos de Capital IQ integrados y P/E calculado!")
        else:
            print("   ⚠️ ERROR: El Excel no tiene columnas 'date' o 'tic'. Revisa el limpiador.")

    except FileNotFoundError:
        print("   ⚠️ AVISO: No se encontró 'tus_datos_capiq.xlsx'.")

    print("4. 🌎 Uniendo Macro a cada Acción...")
    # Unimos el VIX y Bonos a cada fila
    df_final = df_prices.merge(df_macro, on='date', how='left')
    
    # Limpieza final
    df_final = df_final.sort_values(['date', 'tic']).ffill().dropna()
    
    # Generar Covarianzas
    df_final = add_covariance_matrix(df_final, lookback=252)
    
    # Indexar numéricamente (Requisito de FinRL)
    df_final.index = df_final.date.factorize()[0]
    
    print(f"5. 🏁 DataFrame Final Listo. Shape: {df_final.shape}")
    return df_final, len(dow_tickers)

# --- PRUEBA RÁPIDA ---
if __name__ == "__main__":
    df_train, stock_dim = prepare_custom_data()
    
    print("\n--- MUESTRA DE DATOS (Verificación) ---")
    cols_a_ver = ['date', 'tic', 'close', 'eps_norm', 'pe_ratio_daily', 'vix']
    # Mostrar solo columnas que existan
    print(df_train[[c for c in cols_a_ver if c in df_train.columns]].tail())

1. 📉 Descargando Precios + Macro de Yahoo...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (43457, 8)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (4526, 8)
2. 📊 Calculando Indicadores Técnicos...
Successfully added technical indicators
3. 🔗 Fusionando con Datos Fundamentales (Capital IQ)...
   ✅ ¡Datos de Capital IQ integrados y P/E calculado!
4. 🌎 Uniendo Macro a cada Acción...
   -> Generando matrices de covarianza (esto toma unos segundos)...
5. 🏁 DataFrame Final Listo. Shape: (35196, 17)

--- MUESTRA DE DATOS (Verificación) ---
           date  tic       close  eps_norm  pe_ratio_daily    vix
1256 2023-12-29  TRV  184.812943      8.16       22.620923  12.45
1256 2023-12-29  UNH  509.297058     17.30       29.422129  12.45
1256 2023-12-29    V  256.552917      6.11       41.920411  12.45
1256 2023-12-29   VZ   33.068073      3.92        8.414268  12.45
1256 2023-12-29  WMT   51.540096      1.81       28.318734  12.45
